# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AMB3TUC7E to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

csv_file = 'https://raw.githubusercontent.com/SwapnaKategaru/Project3/main/heart_failure_clinical_records_dataset.csv'

data = TabularDatasetFactory.from_delimited_files(csv_file)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core.script_run_config import ScriptRunConfig
from azureml.train.hyperdrive import choice
import os

In [6]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.6.2
- scikit-learn=0.21.2
- pip:
  - azureml-defaults 

Writing conda_dependencies.yml


In [7]:
if "training" not in os.listdir():
    os.mkdir("./training")
    
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=100, slack_factor=0.2, slack_amount=None, delay_evaluation=200)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
       "--C": uniform(0.0001, 1),
       "--max_iter": choice(80, 100, 120, 140, 180)
   })

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=".", compute_target='notebook138600', 
                vm_size="STANDARD_DS2_V2", vm_priority='lowpriority', entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(param_sampling, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=4, max_concurrent_runs=4, policy=early_termination_policy, estimator=estimator)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
#TODO: Submit your experiment
from azureml.widgets import RunDetails
remote_run = experiment.submit(hyperdrive_run_config)
RunDetails(remote_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [10]:
import joblib
# Get your best run and save the model from that run.
model = remote_run.get_best_run_by_primary_metric(include_failed=True, include_canceled=True, include_resume_from_runs=True)
remote_run.wait_for_completion(show_output=True)

best_run = remote_run.get_metrics()           
joblib.dump(value=best_run, filename="./outputs.joblib")


RunId: HD_17ad486c-d982-4196-ab77-b0f63d87035e
Web View: https://ml.azure.com/experiments/hyperdrive/runs/HD_17ad486c-d982-4196-ab77-b0f63d87035e?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-138600/workspaces/quick-starts-ws-138600

Execution Summary
RunId: HD_17ad486c-d982-4196-ab77-b0f63d87035e
Web View: https://ml.azure.com/experiments/hyperdrive/runs/HD_17ad486c-d982-4196-ab77-b0f63d87035e?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-138600/workspaces/quick-starts-ws-138600



['./outputs.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
# best_run.register_model(model_name = 'best_automl_model.pkl', model_path = './outputs/')
joblib.load(filename='./outputs.joblib')

{'HD_17ad486c-d982-4196-ab77-b0f63d87035e_0': {'Regularization Strength:': 0.2209843163513509,
  'Max iterations:': 80,
  'Accuracy': 0.6060606060606061},
 'HD_17ad486c-d982-4196-ab77-b0f63d87035e_1': {'Regularization Strength:': 0.730602613100626,
  'Max iterations:': 80,
  'Accuracy': 0.7373737373737373},
 'HD_17ad486c-d982-4196-ab77-b0f63d87035e_3': {'Regularization Strength:': 0.22736611602784512,
  'Max iterations:': 80,
  'Accuracy': 0.6060606060606061},
 'HD_17ad486c-d982-4196-ab77-b0f63d87035e_2': {'Regularization Strength:': 0.7741721416341513,
  'Max iterations:': 100,
  'Accuracy': 0.7373737373737373}}

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [12]:
from azureml.core.model import Model
model = Model.register(model_name='hyperdrive-XX', model_path='./outputs.joblib', workspace=ws)

Registering model hyperdrive-XX


In [30]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [104]:
# %%writefile score.py

import json
import numpy as np
import pandas as pd
import os
from sklearn.externals import joblib
def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'outputs.joblib')
    print("Found model:", os.path.isfile(model_path)) #To check whether the model is actually present on the location we are looking at
    model = joblib.load(model_path)
def run(data):
    try:
        data = np.array(json.loads(data))
        data = pd.DataFrame.from_dict(data)
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

In [97]:
from azureml.core.webservice import AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,  enable_app_insights=True, auth_enabled=True)

from azureml.core.webservice import LocalWebservice, Webservice
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=environment)
# deployment_config = LocalWebservice.deploy_configuration(port=8890)
service = Model.deploy(ws, "test4", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://41bf5940-9a5f-4eeb-870f-448281ef0d68.southcentralus.azurecontainer.io/score
http://41bf5940-9a5f-4eeb-870f-448281ef0d68.southcentralus.azurecontainer.io/swagger.json


In [98]:
primary, secondary = service.get_keys()
print(primary)

SskshYTefn7DUKyKf4sOz9s8OvHp3ebN


In [108]:
import requests
import json

scoring_uri = 'http://41bf5940-9a5f-4eeb-870f-448281ef0d68.southcentralus.azurecontainer.io/score'

key = 'SskshYTefn7DUKyKf4sOz9s8OvHp3ebN'


# Sample data to send to the service
data = {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
                
input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}

headers['Authorization'] = f'Bearer {key}'

resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)


"'dict' object has no attribute 'predict'"


# _ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ __ _ _ _  _   _ _ _ _ _

In [86]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=environment) 

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=1234)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment() 

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 334dffeb2d574f309fa9a978dc0cddf3.azurecr.io
Logging into Docker registry 334dffeb2d574f309fa9a978dc0cddf3.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 334dffeb2d574f309fa9a978dc0cddf3.azurecr.io/azureml/azureml_8a90a8ae4f21dcd7231b9244082417af
 ---> d3a8ee3aad22
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 65e2336932dc
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImQ0YWQ3MjYxLTgzMmQtNDZiMi1iMDkzLTIyMTU2MDAxZGY1YiIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzODYwMCIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzODYwMCIsIndvcmtzcGFjZUlkIjoiMzM0ZGZmZWItMmQ1Ny00ZjMwLTlmYTktYTk3OGRjMGNkZGYzIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 5dcfc8abcad1
 ---> 37144e2a1163
Step 4/5 : RUN mv '/var/azureml-app/tmpmbi9acd0.py' /var/azureml-app/main.py


In [87]:
print('Local service port: {}'.format(local_service.port))

Local service port: 1234


In [106]:
print(local_service.get_logs())


2021-02-11T19:09:02,188232338+00:00 - gunicorn/run 
2021-02-11T19:09:02,189921549+00:00 - iot-server/run 
2021-02-11T19:09:02,189979049+00:00 - nginx/run 
2021-02-11T19:09:02,193975975+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_c6bf3e53b0b53065cb9df10251065eb1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [107]:
import json

sample_input = json.dumps(
    {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
)

local_service.run(sample_input)

'Must pass 2-d input. shape=()'